In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import os

In [ ]:
mlflow_tracking_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'mlruns'))
mlflow.set_tracking_uri("file:\\"+mlflow_tracking_path)

In [ ]:
EXPERIMENT_NAME = "COPD_classifier_experiments"
MODEL_NAME = "COPD_Classifier"

In [ ]:
client = MlflowClient()
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

In [ ]:
runs = client.search_runs(exp.experiment_id,order_by=["metrics.accuracy DESC"])
best_run = runs[0]
best_acc = best_run.data.metrics["accuracy"]
best_run

In [ ]:
run_id = best_run.info.run_id

In [ ]:
model_uri=f"runs:/{run_id}/COPD_Classifier"
registered_model = mlflow.register_model(model_uri,MODEL_NAME)

In [ ]:
model_version = registered_model.version

In [ ]:
prod_models = [m for m in client.get_latest_versions(MODEL_NAME, stages=["Production"])]
if prod_models:
    current_acc = float(prod_models[0].tags.get("accuracy", 0))
    if best_acc > current_acc:
        client.transition_model_version_stage(MODEL_NAME, model_version, "Production", archive_existing_versions=True)
        client.set_model_version_tag(MODEL_NAME, model_version, "accuracy", str(best_acc))
        print(f"Promoted v{model_version} to Production (acc={best_acc:.4f})")
    else:
        print("No promotion — current Production is better.")
else:
    client.transition_model_version_stage(MODEL_NAME, model_version, "Production")
    client.set_model_version_tag(MODEL_NAME, model_version, "accuracy", str(best_acc))
    print(f"First Production model: v{model_version}")